In [1]:
# =========================
# Library
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import gc
import random
from glob import glob
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import warnings
import seaborn as sns
import pickle
import json
import re
import time
import sys
from requests import get
import multiprocessing
import joblib
import Levenshtein
import difflib
from contextlib import contextmanager
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import log_loss
import logging
warnings.filterwarnings('ignore')
import sys
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm,train_cat_classifier
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# =========================
# Constant
# =========================
TRAIN_PATH = "../data/train.csv"
TARGET = "point_of_interest"

In [3]:
# =========================
# Settings
# =========================
exp = "090"
val_fold = 0
if not os.path.exists(f"../output/exp/ex{exp}"):
    os.makedirs(f"../output/exp/ex{exp}")
    os.makedirs(f"../output/exp/ex{exp}/model")
LOGGER_PATH = f"../output/exp/ex{exp}/ex_{exp}.txt"
MODEL_PATH = f"../output/exp/ex{exp}/model"

SEED = 0
CAT_PARAMS = {
   'iterations':5000,
    'learning_rate': 0.1,
    'loss_function':'Logloss',
    'random_seed':SEED,
    'depth': 9,
    'logging_level':'Silent',
    }

fe_list = ["../output/fe/fe066.pkl",
           "../output/fe/fe067.pkl",
           "../output/fe/fe068.pkl",
           "../output/fe/fe069.pkl"]

In [4]:
# =========================
# Functions
# =========================

def calc_loss(y_true, y_pred):
    return  log_loss(y_true, y_pred)


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')
    


# https://www.kaggle.com/code/columbia2131/foursquare-iou-metrics
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame):
    scores = []
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()


In [5]:
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2022-06-18 05:25:57,620 - INFO - logger set up


<RootLogger root (DEBUG)>

In [6]:
train = pd.read_pickle(fe_list[0]) 

In [7]:
for i in range(1,len(fe_list)):
    _train = pd.read_pickle(fe_list[i])
    train = pd.concat([train,_train],axis=1)
    del _train
    gc.collect()
# train["target"] = train[TARGET] == train["near_target"]
# train["target"] = train["target"].astype(int)

In [8]:
drop_cols = ['id', 'name','match','oof', 'oof_rank',
       'address', 'city', 'state', 'zip','country', 'url', 'phone',
       'categories', 'point_of_interest', 'set',
       'near_target', 'near_id', 'near_name', 'near_address', 'near_city',
       'near_state', 'near_zip', 'near_country', 'near_url', 'near_phone',
       'near_categories','target']
features = [i for i in train.columns if i not in drop_cols]

In [10]:
with timer("catboost"):
    drop_cols = []
    feature_importances = pd.DataFrame()
    categorical_features = []
    fold = 0
    with timer(f"fold {fold}"):
        x_train = train[features].values.astype(np.float32)
        y_train = train["target"].values
        print("train:",len(x_train))
        y_pred_valid, y_pred_test, valid_loss, best_iter, model = train_cat_classifier(
                    x_train, y_train, None,None,None,
                    categorical_features=categorical_features,
                    feature_name=features,
                    cat_params=CAT_PARAMS,
                    loss_func=calc_loss,
                )

train: 43494190


2022-06-18 08:33:10,817 - INFO - [fold 0] done in 11067 s
2022-06-18 08:33:10,818 - INFO - [catboost] done in 11067 s


In [11]:
model.save_model(f"../output/exp/ex{exp}/model/model")